In [1]:
import sys
sys.path.append('../')

In [2]:
from env import DataSource, TradeBroker, TradingEnv

In [3]:
d = DataSource("../../Data/DAT_MT_EURUSD_M1_2021.csv", end_index=30)
tb = TradeBroker(d, initial_balance=10, price_history_length=5)
env = TradingEnv(tb)

In [4]:
env.reset()

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [5]:
for i in range(40):
    obs, reward, done, info = env.step(2)
    if done is True: 
        break

2024-03-08 00:19:34,365 - env - DEBUG - Holding at 1.22322
2024-03-08 00:19:34,369 - env - DEBUG - Holding at 1.22315
2024-03-08 00:19:34,372 - env - DEBUG - Holding at 1.22356
2024-03-08 00:19:34,376 - env - DEBUG - Holding at 1.22341
2024-03-08 00:19:34,379 - env - DEBUG - Holding at 1.2237
2024-03-08 00:19:34,382 - env - DEBUG - Holding at 1.2236
2024-03-08 00:19:34,385 - env - DEBUG - Holding at 1.22354
2024-03-08 00:19:34,386 - env - DEBUG - Holding at 1.22339
2024-03-08 00:19:34,388 - env - DEBUG - Holding at 1.22343
2024-03-08 00:19:34,390 - env - DEBUG - Holding at 1.22333
2024-03-08 00:19:34,393 - env - DEBUG - Holding at 1.22333
2024-03-08 00:19:34,395 - env - DEBUG - Holding at 1.22333
2024-03-08 00:19:34,397 - env - DEBUG - Holding at 1.22366
2024-03-08 00:19:34,399 - env - DEBUG - Holding at 1.22392
2024-03-08 00:19:34,402 - env - DEBUG - Holding at 1.2236
2024-03-08 00:19:34,403 - env - DEBUG - Holding at 1.22359
2024-03-08 00:19:34,405 - env - DEBUG - Holding at 1.22398


In [15]:
env.step(1)

(array([[ 1.22444000e+00, -8.16633185e-05],
        [ 1.22435000e+00, -7.35029891e-05],
        [ 1.22454000e+00,  1.55184384e-04],
        [ 1.22414000e+00, -3.26653274e-04]]),
 0,
 True,
 {})